In [4]:
from openai import OpenAI
client = OpenAI(api_key="")

MODEL = "gpt-3.5-turbo-instruct-0914"

def generate_augmented_sentences(sentence: str, temp) -> list:
    _ = False
    while not _:
        try:
            completion = client.completions.create(
                model=MODEL,
                timeout=5,
                prompt = f"Fill in the blanks with information. Do not modify anything else. There must not be any additional output. Return the entire sentence with the filled-in blanks. \n\n{sentence}",
                # messages=[
                #     {
                #     "role": "user",
                #     "content": f"Fill in the blanks with information. Do not modify anything else. There must not be any additional output.\n\n{sentence}"
                #     }
                # ],
                temperature=temp,
                max_tokens=128,
                top_p=0.9,
                frequency_penalty=0,
                presence_penalty=0
            )
            _ = True
        except Exception as e:
            print(e)
    
    # return completion.choices[0].message.content
    return completion.choices[0].text

In [5]:
# Load dataset
import pandas as pd
data = pd.read_csv('data.csv')
data

,id,sentence,target_term,context_points
0,test_0,"Kirby had limited options, since some of the r...",NaN,Kirby had limited options ___ since some of th...
1,test_1,The professor was used as a guinea pig in the ...,NaN,The ___ professor was used as a guinea pig in ...
2,test_2,His friends said he was to fat to ride a skate...,NaN,His ___ friends said he was to fat to ride a s...
3,test_3,She heard a noise outside the home that spooke...,NaN,She heard a noise outside the ___ home that sp...
4,test_4,These young Afro Americans are just criminals.,NaN,These young Afro Americans are just criminals ...
...,...,...,...,...
110,test_110,How can a man live and defend himself with fou...,NaN,How can a man live and defend himself with fou...
111,test_111,Richard stepped into the elevator that would t...,NaN,Richard stepped into the elevator that would t...
112,test_112,Thomas was getting ready to play in the big fo...,NaN,Thomas ___ was getting ready to play in the __...
113,test_113,The only business trips janitors go on is to t...,NaN,The only ___ business trips janitors go on is ...


In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

In [9]:
def process_sentence(row, temp):
    sent_generations = []

    for i in range(20):
        output = generate_augmented_sentences(row['context_points'], temp)
        sent_generations.append(output)

    with open(f"data/{row['id']}.pkl", "wb") as f:
        pickle.dump(sent_generations, f)

In [10]:
from tqdm import tqdm

with tqdm(total=len(data)) as pbar:
    with ThreadPoolExecutor(max_workers=100) as executor:
        futures = [executor.submit(process_sentence, row, 1.5) for _, row in data.iterrows()]
    for _ in as_completed(futures):
        pbar.update(1)

  0%|          | 0/115 [00:00<?, ?it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct-0914 in organization org-OPJXuCSTyjS0P02P0e2kpQwx on tokens per min (TPM): Limit 90000, Used 89870, Requested 183. Please try again in 35ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct-0914 in organization org-OPJXuCSTyjS0P02P0e2kpQwx on tokens per min (TPM): Limit 90000, Used 89841, Requested 185. Please try again in 17ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct-0914 in organization org-OPJXuCSTyjS0P02P0e2kpQwx on tokens per min (TPM): Limit 90000, Used 89968, Requested 192. Please try again in 106ms. Visit https://platform.openai.com/account/r

100%|██████████| 115/115 [00:56<00:00,  2.02it/s]
